# Day 23: 分類器の実装

## Learning Objectives
- 最近傍法（Nearest Neighbor）の実装
- k-近傍法（k-NN）の実装
- 交差検証（Cross Validation）の実装
- 画像認識のための分類器を構築する

---

# Part 1: Theory (2 hours)

## 23.1 画像認識と分類問題

### 画像認識の基本的な考え方

画像認識は、画像から特徴を抽出し、その特徴に基づいて分類を行うタスクです。

**基本的なフロー**:
1. **前処理**: 画像を正規化、リサイズ
2. **特徴抽出**: 画像から特徴ベクトルを生成
3. **分類**: 特徴ベクトルに基づいてクラスを予測
4. **評価**: 予測結果の精度を評価

**特徴量の種類**:
- ピクセル値そのもの
- ヒストグラム統計量
- エッジ情報
- テクスチャ特徴
- 形状特徴

## 23.2 最近傍法（Nearest Neighbor, NN）

最近傍法は最もシンプルな分類アルゴリズムです。

**基本的なアイデア**:
- 予測対象のデータに最も近い学習データのクラスを予測値とする
- 距離が最も近い1つのデータのみを使用する

**距離の測り方**:

1. **ユークリッド距離（L2ノルム）**:
   $$d(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

2. **マンハッタン距離（L1ノルム）**:
   $$d(x, y) = \sum_{i=1}^{n}|x_i - y_i|$$

3. **最大距離（L∞ノルム）**:
   $$d(x, y) = \max_{i}|x_i - y_i|$$

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import random
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import time

### 23.2.1 最近傍法の実装

In [ ]:
def euclidean_distance(x1, x2):
    """ユークリッド距離の計算"""
    return np.sqrt(sum((a - b) ** 2 for a, b in zip(x1, x2)))

def manhattan_distance(x1, x2):
    """マンハッタン距離の計算"""
    return sum(abs(a - b) for a, b in zip(x1, x2))

class NearestNeighbor:
    """最近傍法分類器"""
    
    def __init__(self, distance_metric='euclidean'):
        """初期化
        
        Args:
            distance_metric: 距離測定方法 ('euclidean' or 'manhattan')
        """
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        """学習データを保存"""
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        """予測を実行"""
        predictions = []
        
        for x in X:
            # 距離の計算
            if self.distance_metric == 'euclidean':
                distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
            elif self.distance_metric == 'manhattan':
                distances = [manhattan_distance(x, x_train) for x_train in self.X_train]
            else:
                raise ValueError("Unsupported distance metric")
            
            # 最も近いデータのインデックスを取得
            nearest_idx = np.argmin(distances)
            nearest_label = self.y_train[nearest_idx]
            
            predictions.append(nearest_label)
            
        return predictions
    
    def predict_with_distance(self, X):
        """予測結果と距離を返す（デバッグ用）"""
        predictions = []
        distances_list = []
        
        for x in X:
            if self.distance_metric == 'euclidean':
                distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
            else:
                distances = [manhattan_distance(x, x_train) for x_train in self.X_train]
            
            nearest_idx = np.argmin(distances)
            nearest_label = self.y_train[nearest_idx]
            min_distance = distances[nearest_idx]
            
            predictions.append(nearest_label)
            distances_list.append(min_distance)
            
        return predictions, distances_list

## 23.3 k-近傍法（k-Nearest Neighbors, k-NN）

k-NNは最近傍法の拡張版です。

**改良点**:
- 予測対象のデータからk個の最近傍を考慮
- 多数決によりクラスを決定
- 以下の点で最近傍法より頑健
  - ノイズに強い
  - 複数の近傍から情報を得られる

**多数決の方法**:
1. **単純多数決**: 最も多いクラスを選択
2. **重み付き多数決**: 距離に反比例した重み付け

**kの選択**:
- kが小さい: バリアンスが大きくなりやすい（過学習）
- kが大きい: バイアスが大きくなりやすい（アンダーフィット）
- 一般的にk=3, 5, 7などの奇数を選択

In [ ]:
class KNearestNeighbors:
    """k-近傍法分類器"""
    
    def __init__(self, k=5, distance_metric='euclidean', weighted=False):
        """初期化
        
        Args:
            k: 近傍の数
            distance_metric: 距離測定方法
            weighted: 重み付き多数決を使用するか
        """
        self.k = k
        self.distance_metric = distance_metric
        self.weighted = weighted
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        """学習データを保存"""
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        """予測を実行"""
        predictions = []
        
        for x in X:
            # 距離の計算
            if self.distance_metric == 'euclidean':
                distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
            elif self.distance_metric == 'manhattan':
                distances = [manhattan_distance(x, x_train) for x_train in self.X_train]
            else:
                raise ValueError("Unsupported distance metric")
            
            # 距離に基づいてインデックスをソート
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = [self.y_train[i] for i in nearest_indices]
            nearest_distances = [distances[i] for i in nearest_indices]
            
            # 多数決で予測
            if self.weighted:
                # 重み付き多数決（重み = 1/距離）
                weights = [1.0 / (d + 1e-10) for d in nearest_distances]
                weighted_votes = {}
                for label, weight in zip(nearest_labels, weights):
                    weighted_votes[label] = weighted_votes.get(label, 0) + weight
                prediction = max(weighted_votes, key=weighted_votes.get)
            else:
                # 単純多数決
                prediction = Counter(nearest_labels).most_common(1)[0][0]
            
            predictions.append(prediction)
            
        return predictions
    
    def predict_with_neighbors(self, X):
        """予測結果と近傍情報を返す（デバッグ用）"""
        predictions = []
        neighbors_info = []
        
        for x in X:
            if self.distance_metric == 'euclidean':
                distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
            else:
                distances = [manhattan_distance(x, x_train) for x_train in self.X_train]
            
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = [self.y_train[i] for i in nearest_indices]
            nearest_distances = [distances[i] for i in nearest_indices]
            
            if self.weighted:
                weights = [1.0 / (d + 1e-10) for d in nearest_distances]
                weighted_votes = {}
                for label, weight in zip(nearest_labels, weights):
                    weighted_votes[label] = weighted_votes.get(label, 0) + weight
                prediction = max(weighted_votes, key=weighted_votes.get)
                votes = weighted_votes
            else:
                prediction = Counter(nearest_labels).most_common(1)[0][0]
                votes = Counter(nearest_labels)
            
            predictions.append(prediction)
            neighbors_info.append({
                'indices': nearest_indices,
                'labels': nearest_labels,
                'distances': nearest_distances,
                'votes': votes
            })
            
        return predictions, neighbors_info

## 23.4 交差検証（Cross Validation）

交差検証は、モデルの評価方法の一つです。

**目的**:
- モデルの汎化性能を正確に評価
- データ分割の影響を減らす
- ハイパーパラメータチューニング

**k-fold交差検証の流れ**:
1. データをk個のグループ（fold）に分割
2. k-1グループを学習データ、1グループを検証データとする
3. すべての組み合わせでk回評価
4. 精度の平均値を最終評価値とする

**kの選択**:
- k=10: 最も一般的
- LOOCV（Leave-One-Out-CV, k=n）: 厳密だば計算量大
- k=5: 計算量と精度のバランス

In [ ]:
def cross_validate(X, y, classifier, k_folds=5, **kwargs):
    """k-fold交差検証を実行
    
    Args:
        X: 特徴ベクトル
        y: ラベル
        classifier: 分類器クラス
        k_folds: foldの数
        **kwargs: 分類器のパラメータ
        
    Returns:
        平均精度と各foldの精度リスト
    """
    # データをシャッフル
    indices = list(range(len(X)))
    random.shuffle(indices)
    
    # k-fold分割
    fold_size = len(X) // k_folds
    accuracies = []
    
    for i in range(k_folds):
        # テストデータのインデックス
        test_start = i * fold_size
        test_end = (i + 1) * fold_size if i < k_folds - 1 else len(X)
        test_indices = indices[test_start:test_end]
        
        # 学習データのインデックス
        train_indices = [idx for idx in indices if idx not in test_indices]
        
        # データ分割
        X_train = [X[idx] for idx in train_indices]
        y_train = [y[idx] for idx in train_indices]
        X_test = [X[idx] for idx in test_indices]
        y_test = [y[idx] for idx in test_indices]
        
        # モデルの学習と評価
        model = classifier(**kwargs)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # 精度計算
        accuracy = sum(1 for pred, true in zip(predictions, y_test) if pred == true) / len(y_test)
        accuracies.append(accuracy)
        
        print(f"Fold {i+1}/{k_folds}: Accuracy = {accuracy:.4f}")
    
    # 平均精度
    mean_accuracy = sum(accuracies) / len(accuracies)
    std_accuracy = np.std(accuracies)
    
    print(f"\n平均精度: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    
    return mean_accuracy, std_accuracy, accuracies

def cross_validate_stratified(X, y, classifier, k_folds=5, **kwargs):
    """層化k-fold交差検証（各foldにクラスの分布を保持）"""
    from sklearn.model_selection import StratifiedKFold
    
    # StratifiedKFoldを使用
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    accuracies = []
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        # データ分割
        X_train = [X[i] for i in train_idx]
        y_train = [y[i] for i in train_idx]
        X_test = [X[i] for i in test_idx]
        y_test = [y[i] for i in test_idx]
        
        # モデルの学習と評価
        model = classifier(**kwargs)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # 精度計算
        accuracy = sum(1 for pred, true in zip(predictions, y_test) if pred == true) / len(y_test)
        accuracies.append(accuracy)
        
        print(f"Fold {fold+1}/{k_folds}: Accuracy = {accuracy:.4f}")
    
    # 平均精度
    mean_accuracy = sum(accuracies) / len(accuracies)
    std_accuracy = np.std(accuracies)
    
    print(f"\n平均精度: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    
    return mean_accuracy, std_accuracy, accuracies

## 23.5 特徴量の正規化

k-NNでは、特徴量のスケールが大きく影響します。

**問題点**:
- 距離計算時にスケールの大きい特徴量が支配的になる
- 例: 画素値(0-255) vs エッジ強度(0-1)

**解決策**:
1. **Min-Max正規化**:
   $$x_{\text{norm}} = \frac{x - \min}{\max - \min}$$

2. **Z正規化（標準化）**:
   $$x_{\text{std}} = \frac{x - \mu}{\sigma}$$

In [ ]:
def normalize_features(X, method='min_max'):
    """特徴量の正規化
    
    Args:
        X: 特徴ベクトルリスト
        method: 'min_max' または 'z_score'
        
    Returns:
        正規化された特徴量リスト
    """
    if method == 'min_max':
        # Min-Max正規化
        min_vals = min(min(sample) for sample in X)
        max_vals = max(max(sample) for sample in X)
        
        if max_vals == min_vals:
            return [[0.5 for _ in sample] for sample in X]
        
        range_val = max_vals - min_vals
        return [[(x - min_vals) / range_val for x in sample] for sample in X]
    
    elif method == 'z_score':
        # Z正規化（特徴量ごとに）
        X_array = np.array(X)
        means = X_array.mean(axis=0)
        stds = X_array.std(axis=0)
        
        # 標準偏差が0の場合を処理
        stds[stds == 0] = 1.0
        
        X_normalized = (X_array - means) / stds
        return X_normalized.tolist()
    
    else:
        raise ValueError("Unsupported normalization method")

def normalize_features_per_feature(X, method='min_max'):
    """各特徴量（次元）ごとに正規化"""
    X_array = np.array(X)
    normalized = []
    
    for dim in range(X_array.shape[1]):
        feature = X_array[:, dim]
        
        if method == 'min_max':
            min_val = feature.min()
            max_val = feature.max()
            if max_val == min_val:
                normalized_feature = np.zeros_like(feature)
            else:
                normalized_feature = (feature - min_val) / (max_val - min_val)
        
        elif method == 'z_score':
            mean_val = feature.mean()
            std_val = feature.std()
            if std_val == 0:
                normalized_feature = np.zeros_like(feature)
            else:
                normalized_feature = (feature - mean_val) / std_val
        
        normalized.append(normalized_feature)
    
    # 次元ごとの正規化を結合
    X_normalized = np.column_stack(normalized)
    return X_normalized.tolist()

---

# Part 2: Practice (2 hours)

それでは、学んだ知識を使って手書き数字認識を実装してみましょう！

## Exercise 23.1: 手書き数字データの生成

In [ ]:
def generate_digit_images(num_samples=100, image_size=8):
    """手書き数字（0-9）のサンプル画像を生成
    
    Returns:
        images: 画像データ（2次元リスト）のリスト
        labels: 数字ラベル（0-9）
    """
    images = []
    labels = []
    
    for digit in range(10):
        digit_samples = num_samples // 10
        
        for _ in range(digit_samples):
            # 数字の画像を生成
            image = [[0 for _ in range(image_size)] for _ in range(image_size)]
            
            # 簡単な数字のパターン
            if digit == 0:  # 0
                for i in range(1, image_size-1):
                    for j in range(1, image_size-1):
                        if (i in [1, image_size-2] or j in [1, image_size-2]):
                            image[i][j] = random.randint(150, 255)
            elif digit == 1:  # 1
                for i in range(image_size):
                    if i != 0 and i != image_size-1:
                        image[i][image_size//2] = random.randint(150, 255)
                    if image_size//2 != 0 and image_size//2 != image_size-1:
                        image[image_size//2][i] = random.randint(150, 255)
            elif digit == 2:  # 2
                # 上半分
                for j in range(image_size):
                    image[1][j] = random.randint(150, 255)
                    if j < image_size-2:
                        image[2][j] = random.randint(150, 255)
                # 中間
                for i in range(1, image_size-1):
                    if i != image_size//2:
                        image[i][image_size-3] = random.randint(150, 255)
                # 下半分
                for j in range(image_size-1, -1, -1):
                    image[image_size-2][j] = random.randint(150, 255)
                    if j > 1:
                        image[image_size-3][j] = random.randint(150, 255)
            elif digit == 3:  # 3
                # 上半分
                for j in range(image_size):
                    image[1][j] = random.randint(150, 255)
                # 右側
                for i in range(1, image_size-1):
                    image[i][image_size-2] = random.randint(150, 255)
                # 下半分
                for j in range(image_size):
                    image[image_size-2][j] = random.randint(150, 255)
                # 中の横棒
                for j in range(2, image_size-2):
                    image[image_size//2][j] = random.randint(150, 255)
            elif digit == 4:  # 4
                # 左側
                for i in range(1, image_size-1):
                    image[i][1] = random.randint(150, 255)
                # 中央縦棒
                for i in range(1, image_size-1):
                    image[i][image_size//2] = random.randint(150, 255)
                # 上横棒
                for j in range(1, image_size//2+1):
                    image[1][j] = random.randint(150, 255)
                # 下横棒
                for j in range(image_size//2, image_size-1):
                    image[image_size//2][j] = random.randint(150, 255)
            elif digit == 5:  # 5
                # 上半分
                for j in range(image_size):
                    image[1][j] = random.randint(150, 255)
                    if j > 1:
                        image[2][j] = random.randint(150, 255)
                # 左側
                for i in range(1, image_size-1):
                    if i != image_size//2:
                        image[i][1] = random.randint(150, 255)
                # 下半分
                for j in range(image_size-1, -1, -1):
                    image[image_size-2][j] = random.randint(150, 255)
                    if j < image_size-2:
                        image[image_size-3][j] = random.randint(150, 255)
            elif digit == 6:  # 6
                # 外枠
                for j in range(image_size):
                    image[1][j] = image[image_size-2][j] = random.randint(150, 255)
                for i in range(2, image_size-2):
                    image[i][1] = image[i][image_size-2] = random.randint(150, 255)
                # 内部横棒
                for j in range(3, image_size-2):
                    image[image_size//2][j] = random.randint(150, 255)
            elif digit == 7:  # 7
                # 上横棒
                for j in range(image_size):
                    image[1][j] = random.randint(150, 255)
                # 斜め
                for i in range(2, image_size-1):
                    j = image_size - 2 - (i - 2)
                    if j >= 1:
                        image[i][j] = random.randint(150, 255)
                # 下部分
                image[image_size-1][1] = random.randint(150, 255)
            elif digit == 8:  # 8
                # 上下円
                for i in [1, image_size-2]:
                    for j in range(1, image_size-1):
                        if i == 1 or i == image_size-2:
                            image[i][j] = random.randint(150, 255)
                # 左右円
                for i in range(2, image_size-2):
                    for j in [1, image_size-2]:
                        image[i][j] = random.randint(150, 255)
                # 中の横棒
                for j in range(2, image_size-2):
                    image[image_size//2][j] = random.randint(150, 255)
            elif digit == 9:  # 9
                # 上半分
                for j in range(image_size):
                    image[1][j] = random.randint(150, 255)
                # 右側
                for i in range(1, image_size-1):
                    if i != image_size//2:
                        image[i][image_size-2] = random.randint(150, 255)
                # 下半分
                for j in range(image_size):
                    image[image_size-2][j] = random.randint(150, 255)
                # 中の横棒
                for j in range(2, image_size-2):
                    image[image_size//2][j] = random.randint(150, 255)
            
            # ノイズを追加
            for i in range(image_size):
                for j in range(image_size):
                    if image[i][j] == 0 and random.random() < 0.05:
                        image[i][j] = random.randint(50, 100)
                    elif image[i][j] > 0 and random.random() < 0.1:
                        image[i][j] = random.randint(0, 50)
            
            images.append(image)
            labels.append(digit)
    
    return images, labels

# データの生成と可視化
images, labels = generate_digit_images(num_samples=500, image_size=8)
print(f"生成データ: {len(images)}枚")
print(f"ラベル分布: {Counter(labels)}")

# サンプル画像の表示
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(f"Digit: {labels[i]}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## Exercise 23.2: 画像から特徴量を抽出

In [ ]:
def extract_simple_features(image):
    """画像から簡単な特徴量を抽出
    
    Args:
        image: 2D画像データ（H×W）
        
    Returns:
        特徴量ベクトル
    """
    features = []
    
    # 1. 平均輝度
    features.append(sum(sum(row) for row in image) / (len(image) * len(image[0])))
    
    # 2. 輝度の分散
    mean_brightness = features[0]
    variance = sum(sum((pixel - mean_brightness) ** 2 for pixel in row) for row in image) / (len(image) * len(image[0]))
    features.append(variance)
    
    # 3. 非ゼロピクセルの割合
    non_zero_pixels = sum(1 for row in image for pixel in row if pixel > 0)
    features.append(non_zero_pixels / (len(image) * len(image[0])))
    
    # 4. 各象限の平均輝度
    height, width = len(image), len(image[0])
    mid_h, mid_w = height // 2, width // 2
    
    # 左上象限
    q1 = sum(sum(image[i][j] for j in range(mid_w)) for i in range(mid_h)) / (mid_h * mid_w)
    features.append(q1)
    
    # 右上象限
    q2 = sum(sum(image[i][j] for j in range(mid_w, width)) for i in range(mid_h)) / (mid_h * (width - mid_w))
    features.append(q2)
    
    # 左下象限
    q3 = sum(sum(image[i][j] for j in range(mid_w)) for i in range(mid_h, height)) / ((height - mid_h) * mid_w)
    features.append(q3)
    
    # 右下象限
    q4 = sum(sum(image[i][j] for j in range(mid_w, width)) for i in range(mid_h, height)) / ((height - mid_h) * (width - mid_w))
    features.append(q4)
    
    # 5. 水平方向のヒストグラム（3バケット）
    hist_h = [0, 0, 0]
    bucket_w = width // 3
    for i in range(height):
        for j in range(width):
            bucket = j // bucket_w
            hist_h[bucket] += image[i][j]
    
    # 平均化
    for i in range(3):
        hist_h[i] /= (height * (bucket_w if i < 2 else width - 2*bucket_w))
        features.append(hist_h[i])
    
    # 6. 垂直方向のヒストグラム（3バケット）
    hist_v = [0, 0, 0]
    bucket_h = height // 3
    for i in range(height):
        for j in range(width):
            bucket = i // bucket_h
            hist_v[bucket] += image[i][j]
    
    # 平均化
    for i in range(3):
        hist_v[i] /= ((bucket_h if i < 2 else height - 2*bucket_h) * width)
        features.append(hist_v[i])
    
    # 7. エッジ密度（簡易版）
    edge_pixels = 0
    for i in range(1, height-1):
        for j in range(1, width-1):
            # ラプラシアン演算子の簡易版
            laplacian = (image[i-1][j] + image[i+1][j] + 
                        image[i][j-1] + image[i][j+1] - 
                        4 * image[i][j])
            if abs(laplacian) > 50:
                edge_pixels += 1
    
    features.append(edge_pixels / ((height-2) * (width-2)))
    
    return features

# 全画像から特徴量を抽出
print("特徴量抽出中...")
X = [extract_simple_features(img) for img in images]
y = labels
print(f"特徴量の次元: {len(X[0])}")
print(f"特徴量サンプル: {X[0][:5]}...")

## Exercise 23.3: 最近傍法の実装と評価

In [ ]:
# データを学習用とテスト用に分割
split_ratio = 0.8
split_idx = int(len(X) * split_ratio)

X_train = X[:split_idx]
y_train = y[:split_idx]
X_test = X[split_idx:]
y_test = y[split_idx:]

print(f"学習データ: {len(X_train)}サンプル")
print(f"テストデータ: {len(X_test)}サンプル")

# 最近傍法の学習と評価
print("\n=== 最近傍法（Euclidean距離） ===")
nn_model = NearestNeighbor(distance_metric='euclidean')
nn_model.fit(X_train, y_train)

# テストデータの予測
y_pred = nn_model.predict(X_test)
accuracy = sum(1 for pred, true in zip(y_pred, y_test) if pred == true) / len(y_test)

print(f"テスト精度: {accuracy:.4f}")

# マンハッタン距離での評価
print("\n=== 最近傍法（Manhattan距離） ===")
nn_model_manhattan = NearestNeighbor(distance_metric='manhattan')
nn_model_manhattan.fit(X_train, y_train)

y_pred_manhattan = nn_model_manhattan.predict(X_test)
accuracy_manhattan = sum(1 for pred, true in zip(y_pred_manhattan, y_test) if pred == true) / len(y_test)

print(f"テスト精度: {accuracy_manhattan:.4f}")

# 予測結果の可視化
def plot_predictions(X_test, y_test, y_pred, title=""):
    """予測結果を表示"""
    correct = 0
    
    # 予測結果のサンプルを表示
    fig, axes = plt.subplots(3, 4, figsize=(12, 9))
    axes = axes.flatten()
    
    for i in range(min(12, len(X_test))):
        # 元の画像を再構築（元のimagesから取得）
        original_index = split_idx + i
        if original_index < len(images):
            axes[i].imshow(images[original_index], cmap='gray')
        
        # 真のラベルと予測ラベル
        true_label = y_test[i]
        pred_label = y_pred[i]
        
        # 色で正誤を表示
        if true_label == pred_label:
            color = 'green'
            correct += 1
        else:
            color = 'red'
        
        axes[i].set_title(f"True: {true_label}\nPred: {pred_label}", color=color)
        axes[i].axis('off')
    
    plt.suptitle(f"{title} (Accuracy: {correct/len(y_test):.2%})")
    plt.tight_layout()
    plt.show()

# 結果の表示
plot_predictions(X_test, y_test, y_pred, "最近傍法（Euclidean距離）")

## Exercise 23.4: k-NNの実装とkの最適化

In [ ]:
# k-NNのk値を変えて評価
k_values = [1, 3, 5, 7, 9]
k_accuracies = []

print("=== k-NNのk値による精度比較 ===")
for k in k_values:
    model = KNearestNeighbors(k=k, distance_metric='euclidean', weighted=False)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = sum(1 for pred, true in zip(y_pred, y_test) if pred == true) / len(y_test)
    k_accuracies.append(accuracy)
    print(f"k={k}: {accuracy:.4f}")

# k値と精度の関係を可視化
plt.figure(figsize=(8, 5))
plt.plot(k_values, k_accuracies, 'bo-')
plt.xlabel('k (Number of Neighbors)')
plt.ylabel('Accuracy')
plt.title('k-NN: Effect of k on Accuracy')
plt.grid(True)
plt.show()

# 最適なkで予測
best_k = k_values[np.argmax(k_accuracies)]
print(f"\n最適なk値: {best_k}")
print(f"最高精度: {max(k_accuracies):.4f}")

# 重み付きk-NNの評価
print("\n=== 重み付きk-NN（Euclidean距離） ===")
knn_weighted = KNearestNeighbors(k=best_k, distance_metric='euclidean', weighted=True)
knn_weighted.fit(X_train, y_train)
y_pred_weighted = knn_weighted.predict(X_test)
accuracy_weighted = sum(1 for pred, true in zip(y_pred_weighted, y_test) if pred == true) / len(y_test)

print(f"テスト精度: {accuracy_weighted:.4f}")

# 重み付きと非重み付きの比較
plot_predictions(X_test, y_test, y_pred_weighted, f"重み付きk-NN (k={best_k})")

## Exercise 23.5: 特徴量正規化の影響

In [ ]:
# 正規化なしでの精度
print("=== 正規化なし ===")
nn_model = NearestNeighbor(distance_metric='euclidean')
nn_model.fit(X_train, y_train)
y_pred = nn_model.predict(X_test)
accuracy_raw = sum(1 for pred, true in zip(y_pred, y_test) if pred == true) / len(y_test)
print(f"テスト精度: {accuracy_raw:.4f}")

# Min-Max正規化
print("\n=== Min-Max正規化 ===")
X_train_normalized = normalize_features(X_train, method='min_max')
X_test_normalized = normalize_features(X_test, method='min_max')

nn_model_norm = NearestNeighbor(distance_metric='euclidean')
nn_model_norm.fit(X_train_normalized, y_train)
y_pred_norm = nn_model_norm.predict(X_test_normalized)
accuracy_norm = sum(1 for pred, true in zip(y_pred_norm, y_test) if pred == true) / len(y_test)
print(f"テスト精度: {accuracy_norm:.4f}")

# Z正規化
print("\n=== Z正規化 ===")
X_train_znorm = normalize_features(X_train, method='z_score')
X_test_znorm = normalize_features(X_test, method='z_score')

nn_model_znorm = NearestNeighbor(distance_metric='euclidean')
nn_model_znorm.fit(X_train_znorm, y_train)
y_pred_znorm = nn_model_znorm.predict(X_test_znorm)
accuracy_znorm = sum(1 for pred, true in zip(y_pred_znorm, y_test) if pred == true) / len(y_test)
print(f"テスト精度: {accuracy_znorm:.4f}")

# 結果の比較
plt.figure(figsize=(10, 6))
methods = ['Raw', 'Min-Max', 'Z-Score']
accuracies = [accuracy_raw, accuracy_norm, accuracy_znorm]
colors = ['blue', 'green', 'red']

bars = plt.bar(methods, accuracies, color=colors)
plt.ylabel('Accuracy')
plt.title('Effect of Feature Normalization')
plt.ylim(0, 1)

# バーに値を表示
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{acc:.3f}', ha='center', va='bottom')

plt.show()

## Exercise 23.6: 交差検証によるモデル評価

In [ ]:
# 交差検証の実行
print("=== 5-fold交差検証 ===")
print("最近傍法（Euclidean距離）:")
nn_cv_mean, nn_cv_std, nn_cv_accs = cross_validate(
    X, y, NearestNeighbor, k_folds=5, distance_metric='euclidean'
)

print("\nk-NN（k=5, Euclidean）:")
knn_cv_mean, knn_cv_std, knn_cv_accs = cross_validate(
    X, y, KNearestNeighbors, k_folds=5, k=5, distance_metric='euclidean'
)

print("\n層化交差検証（sklearn使用）:")
knn_strat_mean, knn_strat_std, knn_strat_accs = cross_validate_stratified(
    X, y, KNearestNeighbors, k_folds=5, k=5, distance_metric='euclidean'
)

# 交差検証結果の可視化
plt.figure(figsize=(10, 6))
plt.boxplot([nn_cv_accs, knn_cv_accs, knn_strat_accs], 
            labels=['NN (Euclidean)', 'k-NN (k=5)', 'k-NN (Stratified)'])
plt.ylabel('Accuracy')
plt.title('Cross-Validation Results Comparison')
plt.grid(True, alpha=0.3)
plt.show()

# 各モデルの性能比較
models = ['NN (Raw)', 'NN (Min-Max)', 'NN (Z-Score)', 
         'k-NN (k=3)', 'k-NN (k=5)', 'k-NN (Weighted)',
         '5-fold CV (NN)', '5-fold CV (k-NN)']
all_accuracies = [
    accuracy_raw,
    accuracy_norm,
    accuracy_znorm,
    max([k_accuracies[i] for i, k in enumerate(k_values) if k == 3]),  # k=3の精度
    max(k_accuracies),  # k=5の精度
    accuracy_weighted,
    nn_cv_mean,
    knn_cv_mean
]

plt.figure(figsize=(12, 8))
bars = plt.bar(range(len(models)), all_accuracies, color='skyblue')
plt.xticks(range(len(models)), models, rotation=45, ha='right')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.ylim(0, 1)

# バーに値を表示
for bar, acc in zip(bars, all_accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Exercise 23.7: 混同行列の分析

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title=""):
    """混同行列を表示"""
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    
    # 混同行列の計算
    cm = confusion_matrix(y_true, y_pred)
    
    # ヒートマップで表示
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=range(10), yticklabels=range(10))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix\n{title}')
    plt.show()
    
    # 各クラスの精度を計算
    class_accuracies = []
    for i in range(10):
        if cm[i, :].sum() > 0:
            class_acc = cm[i, i] / cm[i, :].sum()
        else:
            class_acc = 0
        class_accuracies.append(class_acc)
    
    print(f"\n各クラスの精度:")
    for digit, acc in enumerate(class_accuracies):
        print(f"Digit {digit}: {acc:.3f}")
    
    # 最も間違いやすいペアを特定
    errors = []
    for i in range(10):
        for j in range(10):
            if i != j and cm[i, j] > 0:
                errors.append((i, j, cm[i, j]))
    
    errors.sort(key=lambda x: x[2], reverse=True)
    print("\n最も多い間違い:")
    for true_pred, pred, count in errors[:5]:
        print(f"Digit {true_pred} → {pred}: {count}回")

# 最も良いモデルの混同行列を表示
print("\n=== 最も性能の良いモデルの混同行列 ===")
if nn_cv_mean > knn_cv_mean:
    # 最近傍法が良い場合
    best_model = NearestNeighbor(distance_metric='euclidean')
    model_name = "Nearest Neighbor (Euclidean)"
else:
    # k-NNが良い場合
    best_model = KNearestNeighbors(k=best_k, distance_metric='euclidean')
    model_name = f"k-NN (k={best_k})"

best_model.fit(X_train, y_train)
best_predictions = best_model.predict(X_test)

plot_confusion_matrix(y_test, best_predictions, model_name)

## 追加課題：パフォーマンス分析

In [ ]:
# 予測時間の計測
import time

def measure_prediction_time(model, X_test, num_trials=10):
    """予測にかかる時間を計測"""
    times = []
    for _ in range(num_trials):
        start_time = time.time()
        model.predict(X_test)
        end_time = time.time()
        times.append(end_time - start_time)
    return np.mean(times), np.std(times)

print("=== モデルのパフォーマンス分析 ===")

# 各モデルの予測時間
models_to_test = [
    (NearestNeighbor(distance_metric='euclidean'), "NN (Euclidean)"),
    (NearestNeighbor(distance_metric='manhattan'), "NN (Manhattan)"),
    (KNearestNeighbors(k=1), "k-NN (k=1)"),
    (KNearestNeighbors(k=3), "k-NN (k=3)"),
    (KNearestNeighbors(k=5), "k-NN (k=5)"),
    (KNearestNeighbors(k=5, weighted=True), "k-NN (Weighted)")
]

model_names = []
mean_times = []
std_times = []
accuracies = []

for model, name in models_to_test:
    # 学習
    model.fit(X_train, y_train)
    
    # 精度計算
    pred = model.predict(X_test)
    acc = sum(1 for p, t in zip(pred, y_test) if p == t) / len(y_test)
    
    # 時間計測
    mean_time, std_time = measure_prediction_time(model, X_test)
    
    model_names.append(name)
    accuracies.append(acc)
    mean_times.append(mean_time)
    std_times.append(std_time)
    
    print(f"{name}:")
    print(f"  精度: {acc:.4f}")
    print(f"  予測時間: {mean_time*1000:.2f} ± {std_time*1000:.2f} ms")
    print()

# パレートフロントの可視化（精度 vs 時間）
plt.figure(figsize=(10, 6))
scatter = plt.scatter(mean_times, accuracies, 
                     c=range(len(model_names)), cmap='viridis', s=100)

for i, name in enumerate(model_names):
    plt.annotate(name, (mean_times[i], accuracies[i]), 
                 xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel('Prediction Time (seconds)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Prediction Time Trade-off')
plt.colorbar(scatter, label='Model Index')
plt.grid(True, alpha=0.3)
plt.show()

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] 最近傍法（NN）の概念と実装方法を理解した
- [ ] k-NNの概念と多数決の仕組みを理解した
- [ ] 距離測定方法（Euclidean, Manhattan）の違いを理解した
- [ ] k値の選択がモデル性能に与える影響を理解した

## 特徴量と前処理
- [ ] 特徴量抽出の重要性を理解した
- [ ] 正規化（Min-Max, Z-score）の目的と効果を理解した
- [ ] なぜk-NNでは特徴量のスケールが重要なのか理解した

## 評価方法
- [ ] 交差検証の概念と目的を理解した
- [ ] k-fold交差検証の実装方法を理解した
- [ ] 混同行列の読み方を理解した
- [ ] 各クラスの精度と全体精度の違いを理解した

## 実践力
- [ ] 手書き数字認識を実装した
- [ ] 異なるパラメータでモデル性能を比較した
- [ ] 正規化の影響を分析した
- [ ] 混同行列を使って誤分類パターンを分析した
- [ ] モデルのパフォーマンス（精度 vs 速度）を評価した

---

**お疲れ様でした！** Day 23はこれで終了です。

次回（Day 24）は、モデルの評価結果に基づいて改善策を講じ、より高性能な分類器を構築します。

復習課題：
1. k-NNの計算量がO(n)であることを証明する（nは学習データ数）
2. 近傍数kを最適化する方法を考えてみる
3. どのような特徴量を追加すると精度が向上するか考えてみる